In [1]:
from knossos_utils import knossosdataset, skeleton
import matplotlib.pyplot as plt
import numpy as np
import neuroglancer
%matplotlib notebook

In [2]:
knossos_cube = '/media/disk_trantor_2/data_raf/2018_04_03_WholeBrainMRI1_retake_newfocus/recon_flatcorr_1x/recon_crop8_new_knossos/mag1/knossos.conf'
# knossos_anno = '/home/hanyu/Trantor_2/data_katrina/test_space/annotation-20181106T1033.000.k.zip'
# knossos_anno = '/media/disk_trantor_2/data_katrina/KN_recon_dendrites/annotation-20181011T1135.269.k.zip'
knossos_anno = '/media/disk_trantor_2/data_katrina/KN_WholeBrainMRI1_skeleton&seg/annotation-20181108T1119.129.k.zip'
mag = 1
kd = knossosdataset.KnossosDataset()
kd.initialize_from_knossos_path(knossos_cube)

box_size = [1000, 1000, 1000]
box_offset = [4000, 4000, 4000]

cube = kd.from_raw_cubes_to_matrix(
  size=box_size, 
  offset=box_offset)
anno = kd.from_kzip_to_matrix(
  path=knossos_anno,
  size=box_size,
  offset=box_offset,
  mag=mag, 
  verbose=False)





/home/hanyu/anaconda2/envs/py36/lib/python3.6/site-packages/knossos_utils/knossosdataset.py:1087: UserWarning: You are using implicit channel selection. This possibility will soon be removed. Please call set_channel() before reading or writing data using KnossosDataset.
  warnings.warn('You are using implicit channel selection. This possibility will soon be removed.'


Progress: finished
Speed: 1081.620 MB or MPix /s, time 0.9245390892028809
Progress: 100.00%



In [3]:
ids = np.unique(anno[...])

In [16]:
print(cube.shape, cube.dtype)
print(anno.shape, anno.dtype)



(1000, 1000, 1000) uint8
(1000, 1000, 1000) uint64


In [15]:
skeletons = neuroglancer.skeleton.Skeleton(
  vertex_positions=np.random.randint(0,1000, size=(100,3)),
  edges=[(i,j) for i,j in zip(range(0,99),range(1,100))]
)
voxel_size = np.array([10,10,10])
class SkeletonSource(neuroglancer.skeleton.SkeletonSource):
    def __init__(self):
        super(SkeletonSource, self).__init__()
        self.vertex_attributes['affinity'] = neuroglancer.skeleton.VertexAttributeInfo(
            data_type=np.float32,
            num_components=1,
        )
        self.vertex_attributes['affinity2'] = neuroglancer.skeleton.VertexAttributeInfo(
            data_type=np.float32,
            num_components=1,
        )

    def get_skeleton(self, i):
        pos = np.unravel_index(i, shape, order='C')[::-1]
#         vertex_positions = [pos * voxel_size, pos * voxel_size + np.random.randn(3) * 1000]
        vertex_positions=np.random.randint(0,1000, size=(100,3))
        edges=[(i,j) for i,j in zip(range(0,99),range(1,100))]
        
#         edges = [0, 1]
        return neuroglancer.skeleton.Skeleton(
            vertex_positions=vertex_positions,
            edges=edges,
            vertex_attributes=dict(affinity=np.random.rand(2), affinity2=np.random.rand(2)))


In [16]:
print(skeletons)

In [17]:
print(cube.shape, cube.dtype)
print(anno.shape, anno.dtype)
viewer = neuroglancer.Viewer()
with viewer.txn() as s:
#   s.voxel_size = (600, 600, 600)
  s.layers['image'] = neuroglancer.ImageLayer(
    source=neuroglancer.LocalVolume(data=cube, volume_type='image'))
  s.layers['segmentation'] = neuroglancer.SegmentationLayer(
    source=neuroglancer.LocalVolume(data=anno, volume_type='segmentation', mesh_options={'max_quadrics_error':-1e-5}),
    segments=ids)
  s.layers['skeletons'] = neuroglancer.SegmentationLayer(
    source=neuroglancer.LocalVolume(data=anno, skeletons=SkeletonSource()),
    skeleton_shader='void main() { emitRGB(colormapJet(affinity)); }',
    selected_alpha=0,
    not_selected_alpha=0,
    segments=range(0,100)
  )
  
print(viewer.get_viewer_url())

(1000, 1000, 1000) uint8
(1000, 1000, 1000) uint64
http://127.0.0.1:35581/v/5d8450542d4891d8bbbb64b1fdb187d2aa48c2e0/


In [14]:
del viewer

NameError: name 'viewer' is not defined

In [8]:
import os

In [9]:
import dxchange
os.getcwd()

'/home/hanyu/workspace/KLab_Utils/notebooks'

In [37]:

os.chdir('/home/hanyu/workspace/KLab_Utils')
os.mkdir('bkmovie_test')
os.chdir('bkmovie_test')
os.getcwd()

'/home/hanyu/workspace/KLab_Utils/bkmovie_test'

In [38]:
dxchange.write_tiff_stack(cube,'cube/cube')

In [39]:
anno2 = anno.astype('uint8')
dxchange.write_tiff_stack(anno2,'anno/anno')

In [40]:
os.getcwd()

'/home/hanyu/workspace/KLab_Utils/bkmovie_test'

ERROR:tornado.general:Uncaught exception, closing connection.
Traceback (most recent call last):
  File "/home/hanyu/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/iostream.py", line 713, in _handle_events
    self._handle_write()
  File "/home/hanyu/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/iostream.py", line 1063, in _handle_write
    self._write_buffer.advance(num_bytes)
  File "/home/hanyu/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/iostream.py", line 184, in advance
    assert 0 < size <= self._size
AssertionError
ERROR:asyncio:Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/home/hanyu/anaconda2/envs/py36/lib/python3.6/asyncio/events.py", line 145, in _run
    self._callback(*self._args)
  File "/home/hanyu/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 122, in _handle_events
    handler_func(fileobj, events)
  File "/home/hanyu/anaconda2/envs/py36/lib/python3

ERROR:asyncio:Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/home/hanyu/anaconda2/envs/py36/lib/python3.6/asyncio/events.py", line 145, in _run
    self._callback(*self._args)
  File "/home/hanyu/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 122, in _handle_events
    handler_func(fileobj, events)
  File "/home/hanyu/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/stack_context.py", line 300, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/hanyu/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/iostream.py", line 713, in _handle_events
    self._handle_write()
  File "/home/hanyu/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/iostream.py", line 1063, in _handle_write
    self._write_buffer.advance(num_bytes)
  File "/home/hanyu/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/iostream.py", line 184, in advance
    assert 0 < size <= self._size
AssertionEr

ERROR:tornado.general:Uncaught exception, closing connection.
Traceback (most recent call last):
  File "/home/hanyu/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/iostream.py", line 713, in _handle_events
    self._handle_write()
  File "/home/hanyu/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/iostream.py", line 1063, in _handle_write
    self._write_buffer.advance(num_bytes)
  File "/home/hanyu/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/iostream.py", line 184, in advance
    assert 0 < size <= self._size
AssertionError
ERROR:asyncio:Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/home/hanyu/anaconda2/envs/py36/lib/python3.6/asyncio/events.py", line 145, in _run
    self._callback(*self._args)
  File "/home/hanyu/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 122, in _handle_events
    handler_func(fileobj, events)
  File "/home/hanyu/anaconda2/envs/py36/lib/python3